In [1]:
import numpy as np
def calculate_distance(city1, city2) -> float:  
    """Calculates the Euclidean distance between two cities."""  
    x1 = city1[0]  
    y1 = city1[1]
    x2 = city2[0]
    y2 = city2[1]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)  

def drop_current_city(coordinate, cities):
    index = np.where(np.all(cities == coordinate, axis=1))[0]
    
    if len(index) > 0:
        new_cities = np.delete(cities, index[0], axis=0)
    else:
        new_cities = cities.copy()
    
    return new_cities
    
def generate_tour(city_coords: np.ndarray[np.ndarray]) -> tuple[np.ndarray,float]: 
    """Generates an initial TSP tour and its total distance.
    Args:
        city_coords: List of tuples representing the (x, y) coordinates of each city.
    Returns:
        A tuple containing:
        - Array of city indices representing the initial tour.
        - Total distance of the initial tour.
    """

                 
    current_city = 0
    unvisited_cities = city_coords.copy()

    current_city_coordinate = np.array(unvisited_cities[current_city])

    
    unvisited_cities = drop_current_city(current_city_coordinate,unvisited_cities)
    tour = [current_city_coordinate]
    total_distance = 0

    for i in range(len(city_coords)-1):

        prioritys = priority(current_city_coordinate,unvisited_cities)

        current_city = np.argmax(prioritys)  

        total_distance += calculate_distance(current_city_coordinate, unvisited_cities[current_city])

        current_city_coordinate = unvisited_cities[current_city]

        tour.append(current_city_coordinate)

        unvisited_cities = drop_current_city(current_city_coordinate,unvisited_cities)
  
    return tour, total_distance
    
#@funsearch.run
def evaluate(instances: dict) -> float:  
    distances = []  
    
    for name in instances:  

        instance = instances[name]

        city_coords = np.array(instance['city_coords'])

        tour, distance  = generate_tour(city_coords)

        distances.append(distance)

    return -np.mean(distances)

#@funsearch.evolve
def priority(current_city: np.ndarray, unvisited_cities: np.ndarray[np.ndarray]) -> np.ndarray:
    """Improved version of `priority_v1`."""
    x0, y0 = current_city
    num_cities = len(unvisited_cities)
    
    priorities_1 = np.zeros(num_cities)
    priorities_2 = np.zeros(num_cities)
    priorities_3 = np.zeros(num_cities)
    
    for i in range(num_cities):
        x, y = unvisited_cities[i]
        distance = np.sqrt((x0 - x) ** 2 + (y0 - y) ** 2)  
        angle = np.arctan2(y - y0, x - x0)
        
        # Assign priority based on distance
        if distance < 5:
            priority_distance = 10 - distance
        elif distance < 10:
            priority_distance = 5 - (distance - 5) / 2
        else:
            priority_distance = 1 - (distance - 10) / 5
        
        # Assign additional points based on the angle
        if angle < np.pi/8:
            priority_angle = 5
        elif angle < np.pi/4:
            priority_angle = 3
        elif angle < 3*np.pi/8:
            priority_angle = 2
        else:
            priority_angle = 1
        
        # Assign extra points based on the x and y coordinates
        if x < 0:
            priorities_1[i] = priority_distance + priority_angle
            priorities_2[i] = priority_distance
        else:
            priorities_1[i] = priority_distance + priority_angle
            priorities_2[i] = priority_distance
        
        # Assign extra points based on the Euclidean distance from starting city
        euclidean_distance = np.sqrt(x**2 + y**2)
        if euclidean_distance < 5:
            priority_euclidean = 5
        elif euclidean_distance < 10:
            priority_euclidean = 3
        elif euclidean_distance < 15:
            priority_euclidean = 2
        else:
            priority_euclidean = 1
        
        # Assign priority based on the Euclidean distance
        priorities_3[i] = priority_euclidean
    
    priorities = priorities_1 + priorities_2 + priorities_3
    
    return priorities

In [2]:
def solve_tsp_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    city_coords = []
    for line in lines:
        if line.startswith('NODE_COORD_SECTION'):
            break
    for line in lines[lines.index(line) + 1:]:
        if line.startswith('EOF'):
            break
        city_data = line.strip().split()
        city_coords.append([float(city_data[1]), float(city_data[2])])
#        print("test",city_coords)

    tour, total_distance = generate_tour(np.array(city_coords))
    return tour, total_distance

In [3]:
file_path = 'pr226.tsp'
tour, total_distance = solve_tsp_from_file(file_path)

print("Best tour:", tour)
print("Total distance:", total_distance)

Best tour: [array([15625.,  1150.]), array([15625.,  2150.]), array([16375.,  2550.]), array([16375.,  2650.]), array([16375.,  2750.]), array([16375.,  2850.]), array([16375.,  2950.]), array([16375.,  3050.]), array([16375.,  3200.]), array([16375.,  3300.]), array([16375.,  3400.]), array([16375.,  3500.]), array([16375.,  3600.]), array([16375.,  3700.]), array([16375.,  3800.]), array([13975.,  2200.]), array([13475.,  2050.]), array([13475.,  1900.]), array([13475.,  1750.]), array([13475.,  1600.]), array([13475.,  1200.]), array([13625.,  1200.]), array([13925.,  1200.]), array([14025.,  1200.]), array([14125.,  1200.]), array([14425.,  1200.]), array([14525.,  1200.]), array([14625.,  1200.]), array([13975.,  1500.]), array([13325.,  1200.]), array([13175.,  1200.]), array([13100.,  1725.]), array([12675.,  1725.]), array([12675.,  2150.]), array([13100.,  2150.]), array([13125.,  2500.]), array([13125.,  3000.]), array([12025.,  3300.]), array([11425.,  2300.]), array([11425.